# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086 entries, 0 to 2085
Data columns (total 90 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          2086 non-null   int64  
 1   match_id                            2086 non-null   object 
 2   goals                               2086 non-null   float64
 3   assists                             2086 non-null   float64
 4   direct_goal_contributions           2086 non-null   float64
 5   non_penalty_goals                   2086 non-null   float64
 6   penalties_scored                    2086 non-null   float64
 7   penalties_attempted                 2086 non-null   float64
 8   yellow_cards                        2086 non-null   float64
 9   red_cards                           2086 non-null   float64
 10  expected_goals                      2086 non-null   float64
 11  non_penalty_expected_goals          2086 no

In [4]:
combined.head()

,Unnamed: 0,match_id,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,...,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,0,m-00381,0.720489,0.165106,0.885596,0.656287,0.064203,0.076105,0.395800,0.073663,...,6,4,2,5,11,8,2,1,0,0
1,1,m-00382,0.085780,-0.107836,-0.022056,0.044580,0.041200,0.064124,0.226430,-0.000329,...,4,1,7,4,11,9,1,1,0,0
2,2,m-00383,0.411290,0.245491,0.656781,0.298064,0.113226,0.121619,0.429298,0.001779,...,6,9,5,5,9,11,1,2,0,0
3,3,m-00384,-0.085340,-0.143024,-0.228364,-0.078692,-0.006648,-0.006648,-0.068740,0.029937,...,1,4,2,5,9,8,2,1,0,0
4,4,m-00385,-0.240018,0.037276,-0.202742,-0.250471,0.010452,0.017316,-0.037952,-0.034174,...,2,5,3,5,11,12,2,2,0,0


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","direct_goal_contributions","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"penalties_made","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","progressive_passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns


## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[stats_columns])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = 69
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


2024-01-30 15:02:41.956625: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


34/34 [==============================] - 0s 901us/step - loss: 25.5565 - accuracy: 0.4155
Epoch 2/500
34/34 [==============================] - 0s 461us/step - loss: 16.1589 - accuracy: 0.4700
Epoch 3/500
34/34 [==============================] - 0s 471us/step - loss: 13.1561 - accuracy: 0.4700
Epoch 4/500
34/34 [==============================] - 0s 339us/step - loss: 10.5173 - accuracy: 0.4682
Epoch 5/500
34/34 [==============================] - 0s 342us/step - loss: 9.6101 - accuracy: 0.4682
Epoch 6/500
34/34 [==============================] - 0s 358us/step - loss: 8.8662 - accuracy: 0.4688
Epoch 7/500
34/34 [==============================] - 0s 364us/step - loss: 9.0762 - accuracy: 0.4706
Epoch 8/500
34/34 [==============================] - 0s 390us/step - loss: 8.6784 - accuracy: 0.4682
Epoch 9/500
34/34 [==============================] - 0s 316us/step - loss: 8.8682 - accuracy: 0.4706
Epoch 10/500
34/34 [==============================] - 0s 308us/step - loss: 8.7396 - accuracy: 0.46

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 2s 14ms/step
[ 1.5299108   1.2806082  13.060975   11.655723    4.6520877   4.102093
  5.6641326   4.9750013  10.171222   10.217618    1.3594981   1.7408305
  0.02465395  0.07880523]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 1s 2ms/step - loss: 26.2503 - accuracy: 0.3993
Epoch 2/500
34/34 [==============================] - 0s 948us/step - loss: 14.6137 - accuracy: 0.4700
Epoch 3/500
34/34 [==============================] - 0s 951us/step - loss: 12.4239 - accuracy: 0.4670
Epoch 4/500
34/34 [==============================] - 0s 528us/step - loss: 11.1155 - accuracy: 0.4712
Epoch 5/500
34/34 [==============================] - 0s 437us/step - loss: 10.0771 - accuracy: 0.4652
Epoch 6/500
34/34 [==============================] - 0s 775us/step - loss: 9.4399 - accuracy: 0.4712
Epoch 7/500
34/34 [==============================] - 0s 735us/step - loss: 9.3266 - accuracy: 0.4706
Epoch 8/500
34/34 [==============================] - 0s 672us/step - loss: 9.0732 - accuracy: 0.4676
Epoch 9/500
34/34 [==============================] - 0s 410us/step - loss: 8.8896 - accuracy: 0.4694
Epoch 10/500
34/34 [==============================] - 0s 528us/step - loss: 9.0430 - acc

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 2s 8ms/step
[ 1.6637647   1.4734493  13.479072   12.667978    4.347356    3.8975918
  5.8362417   4.5824103  10.3868475  11.405543    1.3275357   1.7879378
 -0.0787254   0.22032213]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 948us/step - loss: 24.6626 - accuracy: 0.4227
Epoch 2/500
34/34 [==============================] - 0s 475us/step - loss: 13.4443 - accuracy: 0.4688
Epoch 3/500
34/34 [==============================] - 0s 430us/step - loss: 11.6884 - accuracy: 0.4706
Epoch 4/500
34/34 [==============================] - 0s 592us/step - loss: 10.5231 - accuracy: 0.4694
Epoch 5/500
34/34 [==============================] - 0s 1ms/step - loss: 9.8660 - accuracy: 0.4700
Epoch 6/500
34/34 [==============================] - 0s 1ms/step - loss: 9.4576 - accuracy: 0.4706
Epoch 7/500
34/34 [==============================] - 0s 2ms/step - loss: 9.1117 - accuracy: 0.4718
Epoch 8/500
34/34 [==============================] - 0s 746us/step - loss: 9.0556 - accuracy: 0.4700
Epoch 9/500
34/34 [==============================] - 0s 1ms/step - loss: 8.8035 - accuracy: 0.4712
Epoch 10/500
34/34 [==============================] - 0s 467us/step - loss: 8.9459 - accuracy: 

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 595us/step
[ 1.3260571   1.0933852  11.909359   14.439938    3.7943022   4.6567063
  5.0156813   5.8966074   9.924254   10.906683    1.2781171   1.7506132
  0.09563942 -0.02634361]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 1s 2ms/step - loss: 25.5457 - accuracy: 0.3765
Epoch 2/500
34/34 [==============================] - 0s 1ms/step - loss: 13.4567 - accuracy: 0.4676
Epoch 3/500
34/34 [==============================] - 0s 2ms/step - loss: 11.6476 - accuracy: 0.4688
Epoch 4/500
34/34 [==============================] - 0s 1ms/step - loss: 10.5095 - accuracy: 0.4694
Epoch 5/500
34/34 [==============================] - 0s 1ms/step - loss: 9.7798 - accuracy: 0.4694
Epoch 6/500
34/34 [==============================] - 0s 1ms/step - loss: 9.3758 - accuracy: 0.4694
Epoch 7/500
34/34 [==============================] - 0s 2ms/step - loss: 9.1322 - accuracy: 0.4694
Epoch 8/500
34/34 [==============================] - 0s 1ms/step - loss: 9.0606 - accuracy: 0.4700
Epoch 9/500
34/34 [==============================] - 0s 1ms/step - loss: 8.9338 - accuracy: 0.4688
Epoch 10/500
34/34 [==============================] - 0s 484us/step - loss: 8.7825 - accuracy: 0.4682
Epo

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 1s 1ms/step
[ 1.2680297   1.5538174  13.376982   14.692254    4.818415    5.423799
  5.208535    6.169543   11.116065   12.998528    2.0597491   2.4344223
  0.17393251  0.17463695]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 1s 603us/step - loss: 26.7528 - accuracy: 0.3927
Epoch 2/500
34/34 [==============================] - 0s 625us/step - loss: 13.5681 - accuracy: 0.4700
Epoch 3/500
34/34 [==============================] - 0s 659us/step - loss: 11.3677 - accuracy: 0.4694
Epoch 4/500
34/34 [==============================] - 0s 418us/step - loss: 10.2589 - accuracy: 0.4694
Epoch 5/500
34/34 [==============================] - 0s 503us/step - loss: 9.6776 - accuracy: 0.4688
Epoch 6/500
34/34 [==============================] - 0s 431us/step - loss: 9.4583 - accuracy: 0.4694
Epoch 7/500
34/34 [==============================] - 0s 574us/step - loss: 9.0199 - accuracy: 0.4676
Epoch 8/500
34/34 [==============================] - 0s 1ms/step - loss: 8.9145 - accuracy: 0.4700
Epoch 9/500
34/34 [==============================] - 0s 892us/step - loss: 8.9795 - accuracy: 0.4694
Epoch 10/500
34/34 [==============================] - 0s 786us/step - loss: 8.7113 - accu

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 1s 1ms/step
[ 2.0352204   1.1950009  16.090733   11.2909      5.05627     4.1545157
  4.98319     4.8944473  10.749755    7.1821747   1.4317734   1.6545936
  0.04783443  0.11433265]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[stats_columns]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  direct_goal_contributions  non_penalty_goals  \
1276  0.366503 -0.022074                   0.344428           0.371299   
1570 -1.117631 -0.519741                  -1.637373          -0.934464   
864  -0.660156  0.036074                  -0.624082          -0.481211   
1382 -0.499839 -0.325765                  -0.825604          -0.535268   
20   -0.390848 -0.137376                  -0.528224          -0.276079   
...        ...       ...                        ...                ...   
1542  0.715649  0.210350                   0.925999           0.618118   
2062 -0.307234 -0.307463                  -0.614697          -0.252090   
479  -0.156351  0.093343                  -0.063009          -0.169494   
561   0.409117  0.230599                   0.639716           0.274980   
1785  0.059156 -0.117956                  -0.058800           0.107958   

      penalties_scored  penalties_attempted  yellow_cards  red_cards  \
1276         -0.004796            -0.06

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
n=15
print(X_train)
pca = PCA(n_components = n, random_state=576)
pca.fit(X)
feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
feature_to_pc_map.to_csv("../feature_to_15_pcs.csv")

dump(pca, '../prediction_pca.bin')
whole_X_pca = pca.transform(whole_x_train)

pca = PCA(n_components = n, random_state=576)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)


feature_to_pc_map

[[ 0.78622664 -0.0438009   0.48421519 ...  1.16024711  1.3415394
   0.17933687]
 [-2.27468291 -1.65931122 -2.12541236 ...  0.44404888 -1.17566338
  -0.02129511]
 [-1.331175    0.14495778 -0.79111497 ...  0.58120935  1.37025823
  -0.59268471]
 ...
 [-0.29211804  0.33086187 -0.05229599 ...  0.20081513  0.96211483
   0.02711332]
 [ 0.87411575  0.77641947  0.87304918 ... -0.30134203  0.28582233
  -0.77632857]
 [ 0.15234919 -0.35504828 -0.04675348 ... -0.37719573  1.56336307
   1.03168733]]


/opt/homebrew/anaconda3/envs/pl-stats/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
0,0.000136,0.000104,0.000240,0.000124,0.000012,0.000015,0.000184,0.000007,0.000137,0.000125,...,0.002729,0.000062,0.000238,0.000759,0.000524,0.000055,0.000023,0.000018,0.000003,0.000001
1,0.000682,0.000437,0.001119,0.000623,0.000058,0.000073,0.000368,0.000016,0.000653,0.000596,...,0.001994,-0.000150,-0.003448,-0.010841,-0.007447,-0.000737,-0.000311,-0.000236,-0.000055,-0.000020
2,0.001283,0.000729,0.002013,0.001217,0.000067,0.000083,0.000057,0.000002,0.001195,0.001130,...,-0.008346,0.000083,0.002452,0.007685,0.005281,0.000495,0.000222,0.000171,0.000034,0.000016
3,-0.003388,-0.001696,-0.005084,-0.002898,-0.000489,-0.000603,-0.002619,-0.000040,-0.003462,-0.002984,...,-0.022046,-0.000097,0.000546,0.002482,0.001946,0.000592,0.000138,0.000099,0.000022,0.000017
4,0.004922,0.001400,0.006321,0.004230,0.000691,0.000915,-0.000342,0.000109,0.004541,0.003815,...,-0.042308,0.000233,0.006762,0.021064,0.014457,0.001783,0.000688,0.000542,0.000089,0.000058
5,0.001895,0.003521,0.005417,0.001437,0.000459,0.000590,-0.003916,-0.000266,0.000948,0.000481,...,-0.166814,-0.000969,0.006185,0.019417,0.013292,0.001597,0.000364,0.000185,0.000153,0.000026
6,0.008392,0.003614,0.012006,0.007836,0.000556,0.000696,-0.003118,-0.000048,0.007118,0.006561,...,-0.029741,0.000112,0.008385,0.030285,0.022038,0.004470,0.001163,0.001062,0.000091,0.000010
7,0.003584,0.004716,0.008300,0.002915,0.000669,0.000883,0.002842,0.000003,0.003502,0.002802,...,0.087013,0.000278,-0.005318,-0.017915,-0.012720,-0.003860,-0.000494,-0.000376,-0.000100,-0.000019
8,0.012618,0.003347,0.015965,0.011015,0.001604,0.001954,-0.001917,-0.000201,0.012650,0.011106,...,0.069969,-0.001557,-0.027158,-0.088635,-0.061872,-0.004794,-0.002968,-0.002426,-0.000403,-0.000139
9,-0.015601,-0.000306,-0.015908,-0.013464,-0.002138,-0.002488,-0.005903,-0.000163,-0.016663,-0.014682,...,0.066071,-0.002052,-0.035486,-0.111191,-0.076086,-0.006363,-0.003695,-0.003006,-0.000533,-0.000157


The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=stats_columns)
df.index.name = "principle_component_number"
df

,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,-0.115536,-0.126231,-0.124972,-0.115847,-0.072023,-0.073602,-0.130381,-0.079620,-0.119778,-0.120726,...,-0.112724,-0.110643,-0.050401,-0.051413,-0.051602,-0.050656,-0.049312,-0.049328,-0.035046,-0.024478
1,-0.102072,-0.073287,-0.094899,-0.096897,-0.096297,-0.098688,0.011329,-0.003874,-0.099206,-0.093603,...,0.046732,0.156004,0.290146,0.292491,0.292014,0.266876,0.285548,0.271346,0.242687,0.172704
2,0.193355,0.084860,0.157875,0.180777,0.199047,0.199893,-0.097320,-0.065227,0.174286,0.160107,...,-0.192886,-0.028397,0.169138,0.169891,0.169720,0.164265,0.165280,0.160036,0.135043,0.088215
3,0.065905,-0.087470,0.006596,-0.005229,0.468532,0.456264,0.057330,0.050381,0.068693,-0.000070,...,0.123833,0.059671,-0.022109,-0.021090,-0.020131,-0.017251,-0.035863,-0.027687,-0.040100,-0.050784
4,-0.186745,0.090846,-0.082380,-0.227407,0.124313,0.116412,-0.052233,-0.282706,-0.193210,-0.234099,...,-0.242055,-0.097643,0.021893,0.031645,0.034810,0.009636,0.035875,0.012251,0.056600,0.133563
5,0.056248,0.134943,0.090651,0.073876,-0.069697,-0.075156,-0.126695,-0.239391,0.013866,0.027854,...,-0.005072,-0.045048,-0.018651,-0.022721,-0.023709,0.007503,-0.046453,0.003351,0.004173,-0.458311
6,-0.028465,-0.089847,-0.054620,-0.035654,0.024886,0.010840,0.081407,-0.195433,-0.006626,-0.009242,...,0.032477,-0.039535,0.099882,0.069146,0.052859,-0.054793,0.104422,0.178091,0.230184,-0.779410
7,-0.045093,0.055013,-0.006475,-0.045794,-0.024638,-0.008609,-0.024580,0.876477,-0.062010,-0.067611,...,-0.128317,-0.024481,0.044066,0.033555,0.028923,0.026299,0.041431,0.091143,-0.017614,-0.257083
8,-0.104140,-0.084301,-0.100686,-0.144958,0.178077,0.153152,-0.071499,0.092105,-0.080673,-0.114868,...,-0.164546,-0.030427,-0.047673,-0.040166,-0.038004,0.139016,-0.055433,-0.087296,0.126896,-0.102715


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(15, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500
67/67 [==============================] - 1s 625us/step - loss: 31.5008 - accuracy: 0.3453
Epoch 2/500
67/67 [==============================] - 0s 666us/step - loss: 21.2710 - accuracy: 0.4526
Epoch 3/500
67/67 [==============================] - 0s 684us/step - loss: 19.4252 - accuracy: 0.4646
Epoch 4/500
67/67 [==============================] - 0s 430us/step - loss: 18.8800 - accuracy: 0.4652
Epoch 5/500
67/67 [==============================] - 0s 624us/step - loss: 18.6656 - accuracy: 0.4658
Epoch 6/500
67/67 [==============================] - 0s 547us/step - loss: 18.5613 - accuracy: 0.4658
Epoch 7/500
67/67 [==============================] - 0s 481us/step - loss: 18.5419 - accuracy: 0.4646
Epoch 8/500
67/67 [==============================] - 0s 2ms/step - loss: 18.3921 - accuracy: 0.4652
Epoch 9/500
67/67 [==============================] - 0s 722us/step - loss: 18.3779 - accuracy: 0.4658
Epoch 10/500
67/67 [==============================] - 0s 495us/step - loss: 18.3878 

#### Testing

In [33]:
y_hat = model.predict(X_test)

14/14 [==============================] - 29s 16ms/step


In [34]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [35]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) == home_shots+away_shots else 0
	total_shots_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots else 0
	total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if np.floor(home_goals_hat) == home_goals and np.floor(away_goals_hat) == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) == home_fouls + away_fouls else 0
	total_fouls_over += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0

	total_corners += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) == home_corners + away_corners else 0
	total_corners_over += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners else 0
	total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0

	goals_over += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0
	goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

	all_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0
	all_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0

In [36]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.07894736842105263
total_shots_over: 0.3684210526315789
total_shots_under: 0.5526315789473685


total_shots_on_target: 0.11004784688995216
total_shots_on_target_over: 0.30861244019138756
total_shots_on_target_under: 0.5813397129186603


total_booking_points: 0.20574162679425836
total_booking_points_over: 0.11961722488038277
total_booking_points_under: 0.6746411483253588


correct_score: 0.11483253588516747


winner: 0.41148325358851673


total_corners: 0.03588516746411483
total_corners_over: 0.04066985645933014
total_corners_under: 0.9234449760765551


total_fouls: 0.009569377990430622
total_fouls_over: 0.009569377990430622
total_fouls_under: 0.9808612440191388


goals_over: 0.24162679425837322
goals_under: 0.5143540669856459


all_over: 0.0
all_under: 0.21052631578947367


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [37]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(15, activation="relu", input_dim=15))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learn_rate),
		loss="mse",
		metrics=["accuracy"])
	

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [38]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [39]:
# define a grid of the hyperparameter search space
hidden_layer_one = [10,20,30]
learn_rate = [1e-2, 1e-3, 1e-4]
dropout = [0.1, 0.2, 0.3]
batch_size = [16, 32, 64]
epochs = [10, 50, 100, 200]
n_h_layers = [1, 2, 3, 4]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [40]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [41]:
# searcher = GridSearchCV(estimator=model, n_jobs=-2, 
# 	param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)

# searchResults = searcher.fit(X_train, y_train)

# bestScore = searchResults.best_score_
# bestParams = searchResults.best_params_
# print(f"[INFO] best score is {bestScore} using {bestParams}")

In [42]:
# tuned_model = searcher.best_params_
# tuned_model

In [43]:
hidden_layer_one = [10]
learn_rate = [1e-4]
dropout = [0.3]
batch_size = [64]
epochs = [10]
n_h_layers = [3]

model = get_mlp_model(hidden_layer_one=10, learn_rate=0.0001, dropout=0.3, n_h_layers=3)
model.fit(X_train, y_train, epochs=500, batch_size=64, )

model.save("../stats_regression_model.h5")
pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/500
27/27 [==============================] - 7s 11ms/step - loss: 52.2678 - accuracy: 0.0983
Epoch 2/500
27/27 [==============================] - 0s 3ms/step - loss: 51.9816 - accuracy: 0.0863
Epoch 3/500
27/27 [==============================] - 0s 1ms/step - loss: 51.7544 - accuracy: 0.1013
Epoch 4/500
27/27 [==============================] - 0s 2ms/step - loss: 51.4478 - accuracy: 0.0917
Epoch 5/500
27/27 [==============================] - 0s 1ms/step - loss: 51.1835 - accuracy: 0.0905
Epoch 6/500
27/27 [==============================] - 0s 1ms/step - loss: 50.8578 - accuracy: 0.0983
Epoch 7/500
27/27 [==============================] - 0s 1ms/step - loss: 50.3861 - accuracy: 0.1019
Epoch 8/500
27/27 [==============================] - 0s 881us/step - loss: 50.0903 - accuracy: 0.0923
Epoch 9/500
27/27 [==============================] - 0s 2ms/step - loss: 49.5870 - accuracy: 0.0887
Epoch 10/500
27/27 [==============================] - 0s 3ms/step - loss: 49.2186 - accuracy: 0.0

27/27 [==============================] - 0s 1ms/step - loss: 15.6270 - accuracy: 0.3118
Epoch 135/500
27/27 [==============================] - 0s 3ms/step - loss: 15.6493 - accuracy: 0.3135
Epoch 136/500
27/27 [==============================] - 0s 2ms/step - loss: 15.4346 - accuracy: 0.3046
Epoch 137/500
27/27 [==============================] - 0s 645us/step - loss: 15.2626 - accuracy: 0.3058
Epoch 138/500
27/27 [==============================] - 0s 915us/step - loss: 15.4690 - accuracy: 0.3195
Epoch 139/500
27/27 [==============================] - 0s 512us/step - loss: 15.4195 - accuracy: 0.3201
Epoch 140/500
27/27 [==============================] - 0s 672us/step - loss: 15.1127 - accuracy: 0.3135
Epoch 141/500
27/27 [==============================] - 0s 558us/step - loss: 15.2904 - accuracy: 0.3147
Epoch 142/500
27/27 [==============================] - 0s 837us/step - loss: 15.4759 - accuracy: 0.3046
Epoch 143/500
27/27 [==============================] - 0s 780us/step - loss: 15.1740

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,pca12,pca13,pca14
0,8.288104,0.777311,-1.209873,0.665553,-2.276134,0.832198,-0.504708,1.478592,0.907176,0.353875,-1.602907,-1.179467,0.373405,0.138952,0.088446
1,2.379497,1.681216,0.442543,1.421353,-0.600274,0.541479,1.045526,-0.472594,0.171478,-0.670914,0.746378,-0.088722,-0.276337,0.588030,0.292223
2,5.688341,-2.198912,-0.469306,1.512644,0.095020,0.113965,0.069563,-0.473177,0.212977,0.167321,0.335991,-0.448720,0.123414,0.067825,-0.249892
3,0.471930,1.518938,-1.722807,-1.272104,0.347487,1.295362,1.363253,1.123444,0.392665,-0.211378,1.265849,0.925626,-0.280154,-0.330837,0.171586
4,0.238480,-0.403603,1.757656,0.719845,0.762521,-1.448240,1.017783,-0.684984,-0.327897,0.575677,-0.144779,0.608889,-0.510947,1.282638,0.519534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,-6.454434,2.497479,1.611876,-0.850944,0.455262,0.158055,-0.146412,-0.346509,0.084559,0.185918,0.103905,-0.235669,-0.221763,-0.069919,-0.077953
2082,-3.201740,-3.344254,1.209223,1.268023,0.393165,-1.379902,-0.723362,-0.489492,1.127527,-0.670462,0.909913,1.368820,0.321264,-0.412702,-0.213257
2083,0.605168,6.699500,-1.910104,-1.699892,1.950689,1.491757,-0.341996,-0.621834,0.426918,0.499227,-0.872600,0.145287,-0.624366,0.320337,-0.340220
2084,2.651443,1.262765,-0.075410,-1.436003,-1.640736,-0.939945,0.613312,-0.405761,0.410406,0.065311,0.370091,-0.641057,-0.395818,0.095717,0.366724


In [44]:
print(len
	  (X_train))

1668


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?